## Clase 4 nov. actividades
- hub de mayor tamaño
- identificar las comunidades, plotearlas de diferente color

In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.signal import coherence, hilbert
import scipy.io

In [2]:
import mne
raw = mne.io.read_raw_edf(r"/Users/Maca/Desktop/Todo todo/UNAM/5º Semestre/Neurociencias-2026-1/S03_datasets/EEG_practica/chb01_01.edf", preload=True)
sfreq_target = 256            
fmin, fmax = 8, 13 

raw.resample(sfreq_target)
raw.filter(1., 45., fir_design='firwin')
data = raw.get_data()
ch_names = raw.info['ch_names']
n_channels = len(ch_names)

coordenadas = r"/Users/Maca/Desktop/Todo todo/UNAM/5º Semestre/Neurociencias-2026-1/S03_datasets/EEG_practica/coordenadas_bipolares_eeg.csv"
coordenadas_df = pd.read_csv(coordenadas)
coordenadas_df.set_index("canal", inplace = True, drop = True)
pos3D = {canal : (coordenadas_df.loc[canal, "x"],
                 coordenadas_df.loc[canal, "y"],
                 coordenadas_df.loc[canal, "z"]) for canal in coordenadas_df.index}

Extracting EDF parameters from /Users/Maca/Desktop/Todo todo/UNAM/5º Semestre/Neurociencias-2026-1/S03_datasets/EEG_practica/chb01_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


/var/folders/48/5mv2c8bn0mn_vl9c6chb0flm0000gp/T/ipykernel_17977/3190415698.py:2: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(r"/Users/Maca/Desktop/Todo todo/UNAM/5º Semestre/Neurociencias-2026-1/S03_datasets/EEG_practica/chb01_01.edf", preload=True)


Sampling frequency of the instance is already 256.0, returning unmodified.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 845 samples (3.301 s)



In [ ]:
# filtrado: MATRIZ DE COHERENCIA ESPECTRAL 
coh_matrix = np.zeros((n_channels, n_channels))
for i in range(n_channels):
    for j in range(n_channels):
        f, Cxy = coherence(data[i], data[j], fs=sfreq_target, nperseg=sfreq_target*2)
        mask = (f >= fmin) & (f <= fmax)
        coh_matrix[i, j] = np.mean(Cxy[mask])

eeg_df = pd.DataFrame(coh_matrix, index=ch_names, columns=ch_names)
np.fill_diagonal(eeg_df.values, 0)

In [ ]:
plt.hist(eeg_df)

In [ ]:
import community.community_louvain as community_louvain
# COMUNIDADES
G_edf_abs = nx.from_numpy_array(np.abs(eeg_df.values))
edf_partition = community_louvain.best_partition(G_edf_abs)
print("Comunidades:", edf_partition)

# HUBS
edf_degrees_abs = dict(G_edf_abs.degree())
edf_avg_degree_abs = np.mean(list(edf_degrees_abs.values()))
edf_hubs_abs = [node for node, degree in edf_degrees_abs.items() if degree > edf_avg_degree_abs]
print("Hubs", edf_hubs_abs)

In [ ]:
# graficar 
plt.figure(figsize = (7, 7))
G_coh = nx.from_pandas_adjacency(eeg_df, create_using = nx.DiGraph())
nx.draw_networkx(G_coh, pos = pos3D, s=edf_hubs_abs, node_color = edf_partition)
plt.show()


plt.show()

prueba commit